# Loading Liabrary and Dataset

In [1]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
data = pd.read_excel('Bank_Transactions.xlsx')

# Formatting Date in required Format

In [2]:
data['Date_Formated'] =  pd.to_datetime(data['VALUE DATE']).dt.strftime('%b-%Y')

# Combine 'WITHDRAWAL AMT' and 'DEPOSIT AMT' into a single 'TRANSACTION AMOUNT' column
data['TRANSACTION AMOUNT'] = data['WITHDRAWAL AMT'].fillna(0) + data['DEPOSIT AMT'].fillna(0)
data.head()

,Account No,TRANSACTION DETAILS,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,Date_Formated,TRANSACTION AMOUNT
0,409000611074',FDRL/INTERNAL FUND TRANSFE,2018-10-01,NaN,500000.00,Oct-2018,500000.00
1,409000611074',Indfor INCOME INDO REMI29091,2018-10-01,2040.00,NaN,Oct-2018,2040.00
2,409000611074',INDO GIBL Indiaforensic STL29091,2018-10-01,333800.00,NaN,Oct-2018,333800.00
3,409000611074',Indfor INCOME INDO REMI30091,2018-10-01,120.00,NaN,Oct-2018,120.00
4,409000611074',INDO GIBL Indiaforensic STL30091,2018-10-01,5000.00,NaN,Oct-2018,5000.00


# Aggregating Transaction Amount for each Account for each Month

In [3]:
# Group by 'Account No' and 'Month', summing 'TRANSACTION AMOUNT'
monthly_transactions = data.groupby(['Account No', 'Date_Formated'])['TRANSACTION AMOUNT'].sum().reset_index()
monthly_transactions.head()

,Account No,Date_Formated,TRANSACTION AMOUNT
0,1196428',Dec-2018,1382227665.94
1,1196428',Feb-2019,425460703.18
2,1196428',Jan-2019,508858056.88
3,1196428',Mar-2019,71073820.41
4,1196428',Nov-2018,1823896265.12


# Applying Pivot

In [4]:
# Pivot the table to have one row per account and 6 columns for the six months
pivot_table = monthly_transactions.pivot(index='Account No', columns='Date_Formated', values='TRANSACTION AMOUNT').reset_index()

In [5]:
#Re-arranging the columns
req_order = ['Account No','Oct-2018', 'Nov-2018', 'Dec-2018','Jan-2019','Feb-2019','Mar-2019']

# Reorganize columns
pivot_table = pivot_table[req_order]
pivot_table = pivot_table.fillna(0)

pivot_table

Date_Formated,Account No,Oct-2018,Nov-2018,Dec-2018,Jan-2019,Feb-2019,Mar-2019
0,1196428',1886306456.59,1823896265.12,1382227665.94,508858056.88,425460703.18,71073820.41
1,1196711',565235354.74,12866821.98,24481699.00,9592454.88,9197185.00,0.00
2,409000362497',262497503.18,138439767.18,118798712.77,138949264.54,79777328.49,22421752.16
3,409000405747',16977846.00,3431011.00,13455799.00,3447857.00,1546542.00,1696058.00
4,409000425051',384981569.00,2539167.00,4987265.00,5074991.00,2303989.00,2535326.00
5,409000438611',860316740.41,881093316.73,695676926.34,359924200.39,308425505.73,45115132.04
6,409000438620',327532710.31,201761533.12,109452685.63,69249745.39,53328227.14,12072563.96
7,409000493201',15489925.63,22088830.42,18263699.11,9095640.50,6313742.04,507361.06
8,409000493210',7457809.12,8517501.72,4758886.22,6392463.52,4967908.28,527218.27
9,409000611074',20085760.00,21584766.00,21018940.00,22912050.00,733571.00,0.00


# Below is the SQL code to do the same

In [ ]:
SELECT 
    Account_No,
	SUM(CASE WHEN EXTRACT(MONTH FROM VALUE_DATE) = 10 THEN COALESCE(WITHDRAWAL_AMT, 0) + COALESCE(DEPOSIT_AMT, 0) ELSE 0 END) AS Oct_2018,
    SUM(CASE WHEN EXTRACT(MONTH FROM VALUE_DATE) = 11 THEN COALESCE(WITHDRAWAL_AMT, 0) + COALESCE(DEPOSIT_AMT, 0) ELSE 0 END) AS Nov_2018,
    SUM(CASE WHEN EXTRACT(MONTH FROM VALUE_DATE) = 12 THEN COALESCE(WITHDRAWAL_AMT, 0) + COALESCE(DEPOSIT_AMT, 0) ELSE 0 END) AS Dec_2018
    SUM(CASE WHEN EXTRACT(MONTH FROM VALUE_DATE) = 1 THEN COALESCE(WITHDRAWAL_AMT, 0) + COALESCE(DEPOSIT_AMT, 0) ELSE 0 END) AS Jan_2019,
    SUM(CASE WHEN EXTRACT(MONTH FROM VALUE_DATE) = 2 THEN COALESCE(WITHDRAWAL_AMT, 0) + COALESCE(DEPOSIT_AMT, 0) ELSE 0 END) AS Feb_2019,
    SUM(CASE WHEN EXTRACT(MONTH FROM VALUE_DATE) = 3 THEN COALESCE(WITHDRAWAL_AMT, 0) + COALESCE(DEPOSIT_AMT, 0) ELSE 0 END) AS Mar_2019,

FROM 
    Data_Table
GROUP BY 
    Account_No;